In [1]:

from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
    save = pickle.load(f)
    train_dataset = save['train_dataset']
    train_labels = save['train_labels']
    valid_dataset = save['valid_dataset']
    valid_labels = save['valid_labels']
    test_dataset = save['test_dataset']
    test_labels = save['test_labels']
    del save  # hint to help gc free up memory
    print('Training set', train_dataset.shape, train_labels.shape)
    print('Validation set', valid_dataset.shape, valid_labels.shape)
    print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


array([[[ 1.,  0.,  0., ...,  0.,  0.,  0.]],

       [[ 1.,  0.,  0., ...,  0.,  0.,  0.]],

       [[ 1.,  0.,  0., ...,  0.,  0.,  0.]],

       ..., 
       [[ 1.,  0.,  0., ...,  0.,  0.,  0.]],

       [[ 1.,  1.,  0., ...,  0.,  0.,  0.]],

       [[ 1.,  0.,  0., ...,  0.,  0.,  0.]]], dtype=float32)

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):   
    dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 0 to [1.0, 0.0, 0.0 ...], 1 to [0.0, 1.0, 0.0 ...]
    labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
    return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
# With gradient descent training, even this much data is prohibitive.
# Subset the training data for faster turnaround.
train_subset = 10000

graph = tf.Graph()
with graph.as_default():

  # Input data.
  # Load the training, validation and test data into constants that are
  # attached to the graph.
  tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
  tf_train_labels = tf.constant(train_labels[:train_subset])
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  # These are the parameters that we are going to be training. The weight
  # matrix will be initialized using random values following a (truncated)
  # normal distribution. The biases get initialized to zero.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  # We multiply the inputs with the weight matrix, and add biases. We compute
  # the softmax and cross-entropy (it's one operation in TensorFlow, because
  # it's very common, and it can be optimized). We take the average of this
  # cross-entropy across all training examples: that's our loss.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  
  # Optimizer.
  # We are going to find the minimum of this loss using gradient descent.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  # These are not part of training, but merely here so that we can report
  # accuracy figures as we train.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [5]:
num_steps = 801

def accuracy(predictions, labels):
    
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

with tf.Session(graph=graph) as session:
  # This is a one-time operation which ensures the parameters get initialized as
  # we described in the graph: random weights for the matrix, zeros for the
  # biases. 
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    # Run the computations. We tell .run() that we want to run the optimizer,
    # and get the loss value and the training predictions returned as numpy
    # arrays.
    _, l, predictions = session.run([optimizer, loss, train_prediction])
    if (step % 100 == 0):
      print('Loss at step %d: %f' % (step, l))
      print('Training accuracy: %.1f%%' % accuracy(
        predictions, train_labels[:train_subset, :]))
      # Calling .eval() on valid_prediction is basically like calling run(), but
      # just to get that one numpy array. Note that it recomputes all its graph
      # dependencies.
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Loss at step 0: 17.191860
Training accuracy: 9.0%
Validation accuracy: 12.1%
Loss at step 100: 2.385466
Training accuracy: 71.8%
Validation accuracy: 70.1%
Loss at step 200: 1.850979
Training accuracy: 75.4%
Validation accuracy: 73.0%
Loss at step 300: 1.590147
Training accuracy: 77.0%
Validation accuracy: 74.1%
Loss at step 400: 1.422201
Training accuracy: 77.9%
Validation accuracy: 74.7%
Loss at step 500: 1.298709
Training accuracy: 78.6%
Validation accuracy: 75.1%
Loss at step 600: 1.201503
Training accuracy: 79.0%
Validation accuracy: 75.2%
Loss at step 700: 1.122139
Training accuracy: 79.4%
Validation accuracy: 75.4%
Loss at step 800: 1.055900
Training accuracy: 79.8%
Validation accuracy: 75.5%
Test accuracy: 83.5%


In [46]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels], stddev=0.1))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [47]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.246664
Minibatch accuracy: 5.5%
Validation accuracy: 34.7%
Minibatch loss at step 500: 0.746279
Minibatch accuracy: 78.9%
Validation accuracy: 81.5%
Minibatch loss at step 1000: 0.596185
Minibatch accuracy: 83.6%
Validation accuracy: 81.5%
Minibatch loss at step 1500: 0.490514
Minibatch accuracy: 87.5%
Validation accuracy: 81.8%
Minibatch loss at step 2000: 0.472146
Minibatch accuracy: 84.4%
Validation accuracy: 81.3%
Minibatch loss at step 2500: 0.756217
Minibatch accuracy: 76.6%
Validation accuracy: 79.6%
Minibatch loss at step 3000: 0.702687
Minibatch accuracy: 81.2%
Validation accuracy: 81.4%
Test accuracy: 88.8%


In [55]:
batch_size = 128
hidden_unit = 1024
import math
graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  #set up hidden layer
  hidden_weight = tf.Variable(
  tf.truncated_normal([image_size * image_size, hidden_unit], stddev=0.1))
  hidden_bias = tf.Variable(tf.zeros([hidden_unit]))
  hidden_layer= tf.nn.relu(tf.matmul(tf_train_dataset, hidden_weight) + hidden_bias)


  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([hidden_unit, num_labels], stddev=0.1))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(hidden_layer, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)

  valid_hidden_layer= tf.nn.relu(tf.matmul(tf_valid_dataset, hidden_weight) + hidden_bias)
  valid_logit = tf.matmul(valid_hidden_layer, weights) + biases
  valid_prediction = tf.nn.softmax(valid_logit)
    
  test_hidden_layer= tf.nn.relu(tf.matmul(tf_test_dataset, hidden_weight) + hidden_bias)
  test_logit = tf.matmul(test_hidden_layer, weights) + biases  
  test_prediction = tf.nn.softmax(test_logit)

In [57]:
num_steps = 6001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 4.942040
Minibatch accuracy: 8.6%
Validation accuracy: 28.4%
Minibatch loss at step 500: 0.529941
Minibatch accuracy: 85.9%
Validation accuracy: 85.1%
Minibatch loss at step 1000: 0.483208
Minibatch accuracy: 87.5%
Validation accuracy: 85.7%
Minibatch loss at step 1500: 0.281987
Minibatch accuracy: 93.0%
Validation accuracy: 87.1%
Minibatch loss at step 2000: 0.271032
Minibatch accuracy: 90.6%
Validation accuracy: 87.7%
Minibatch loss at step 2500: 0.459706
Minibatch accuracy: 86.7%
Validation accuracy: 86.1%
Minibatch loss at step 3000: 0.514814
Minibatch accuracy: 85.2%
Validation accuracy: 88.0%
Minibatch loss at step 3500: 0.494162
Minibatch accuracy: 88.3%
Validation accuracy: 88.5%
Minibatch loss at step 4000: 0.457012
Minibatch accuracy: 83.6%
Validation accuracy: 88.0%
Minibatch loss at step 4500: 0.455823
Minibatch accuracy: 88.3%
Validation accuracy: 88.8%
Minibatch loss at step 5000: 0.196778
Minibatch accuracy: 93.0%
Validation accuracy

In [59]:
batch_size = 128
hidden_unit1 = 1024
hidden_unit2 = 1024
import math
graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  #set up hidden layer 1
  hidden_weight1 = tf.Variable(
  tf.truncated_normal([image_size * image_size, hidden_unit1], stddev=0.1))
  hidden_bias1 = tf.Variable(tf.zeros([hidden_unit1]))
  hidden_layer1= tf.nn.relu(tf.matmul(tf_train_dataset, hidden_weight1) + hidden_bias1)

  #set up hidden layer 2
  hidden_weight2 = tf.Variable(
  tf.truncated_normal([hidden_unit1, hidden_unit2], stddev=0.1))
  hidden_bias2 = tf.Variable(tf.zeros([hidden_unit2]))
  hidden_layer2= tf.nn.relu(tf.matmul(hidden_layer1, hidden_weight2) + hidden_bias2)

  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([hidden_unit2, num_labels], stddev=0.1))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(hidden_layer2, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)

  valid_hidden_layer1= tf.nn.relu(tf.matmul(tf_valid_dataset, hidden_weight1) + hidden_bias1)
  valid_hidden_layer2= tf.nn.relu(tf.matmul(valid_hidden_layer1, hidden_weight2) + hidden_bias2)
  valid_logit = tf.matmul(valid_hidden_layer2, weights) + biases
  valid_prediction = tf.nn.softmax(valid_logit)
    
  test_hidden_layer1= tf.nn.relu(tf.matmul(tf_test_dataset, hidden_weight1) + hidden_bias1)
  test_hidden_layer2= tf.nn.relu(tf.matmul(test_hidden_layer1, hidden_weight2) + hidden_bias2)
  test_logit = tf.matmul(test_hidden_layer2, weights) + biases  
  test_prediction = tf.nn.softmax(test_logit)

In [60]:
num_steps = 6001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 7.239608
Minibatch accuracy: 17.2%
Validation accuracy: 10.0%
Minibatch loss at step 500: 0.514754
Minibatch accuracy: 87.5%
Validation accuracy: 82.4%
Minibatch loss at step 1000: 0.423620
Minibatch accuracy: 86.7%
Validation accuracy: 85.3%
Minibatch loss at step 1500: 0.305939
Minibatch accuracy: 93.0%
Validation accuracy: 86.9%
Minibatch loss at step 2000: 0.274011
Minibatch accuracy: 91.4%
Validation accuracy: 87.5%
Minibatch loss at step 2500: 0.344592
Minibatch accuracy: 89.8%
Validation accuracy: 87.3%
Minibatch loss at step 3000: 0.399579
Minibatch accuracy: 86.7%
Validation accuracy: 88.2%
Minibatch loss at step 3500: 0.320803
Minibatch accuracy: 90.6%
Validation accuracy: 88.2%
Minibatch loss at step 4000: 0.367421
Minibatch accuracy: 85.9%
Validation accuracy: 88.5%
Minibatch loss at step 4500: 0.361342
Minibatch accuracy: 88.3%
Validation accuracy: 87.3%
Minibatch loss at step 5000: 0.178725
Minibatch accuracy: 94.5%
Validation accurac